# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.


In [4]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [6]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [7]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.

It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.


In [8]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [14]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [15]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [16]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [17]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1',
 '/Zyphra/Zonos-v0.1-hybrid',
 '/ValueFX9507/Tifa-Deepsex-14b-CoT-GGUF-Q4',
 '/deepseek-ai/Janus-Pro-7B',
 '/hexgrad/Kokoro-82M',
 '/models',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/deepseek-ai/deepseek-vl2-small',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/simplescaling/s1K',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/HuggingFaceFW/fineweb',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft'

In [18]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o


In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome to Inference Providers on the Hub 🔥
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-R1
Updated
3 days ago
•
2.94M
•
8.31k
Zyphra/Zonos-v0.1-hybrid
Updated
about 1 hour ago
•
874
•
540
ValueFX9507/Tifa-Deepsex-14b-CoT-GGUF-Q4
Upd

In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome to Inference Providers on the Hub 🔥\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\n3 days ago\n•\n2.94M\n•\n8.31k\nZyphra/Zonos-v0.1-hybrid\nUpdated\nabout 1 hour ago\n•\n874\n•\n540\nValueFX9507/Tifa-Deepsex-14b-CoT-GGUF-Q4\nUpdated\nabout 8 hours ago\n•\n43.9k\n•\n442\ndeepseek-ai/Janus-Pro-7B\nUpdat

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

---

## Welcome to Hugging Face!

### The AI Community Building the Future

At Hugging Face, we are at the forefront of Artificial Intelligence (AI) and Machine Learning (ML), providing a platform where the global community collaborates on models, datasets, and applications. Our mission is to democratize AI and make it accessible and beneficial to everyone.

---

## Our Platform

**Discover, Create, Collaborate:**
- **Models**: Explore a vast library of over **1 million models** tailored for various tasks in AI.
- **Datasets**: Access to **250,000+ datasets** that drive innovation in AI and ML.
- **Spaces**: Showcase and share your applications, with **400,000+ applications** already making waves in the community.
- **Enterprise Solutions**: Offering tailored packages for businesses, with advanced features like dedicated support and enterprise-grade security.

---

## Customers

Join a thriving ecosystem with over **50,000 organizations** using Hugging Face, including giants like:
- **Google**
- **Microsoft**
- **Amazon Web Services**
- **Meta**

These organizations collaborate on our platform to enhance their AI capabilities and drive meaningful innovation.

---

## Careers at Hugging Face

We are actively building a team that champions inclusivity, creativity, and collaboration. At Hugging Face, you will have the opportunity to thrive in a culture that values:
- **Innovation**: We believe in pushing boundaries and thinking outside the box.
- **Collaboration**: Work alongside some of the brightest minds in AI and ML.
- **Community**: Engage with a diverse community of developers, researchers, and users.

Explore open positions in various domains and join us to contribute to the future of AI!

---

## Company Culture

At Hugging Face, our culture is centered around:
- **Open Source**: We build tools and technologies that are shared and developed in collaboration with the community.
- **Diversity and Inclusion**: We encourage diverse voices and ideas, ensuring that everyone feels welcome.
- **Continuous Learning**: We foster a learning environment where curiosity is encouraged, and personal growth is valued.

---

## Join Us!

Whether you're a potential customer looking to integrate AI into your projects, an investor interested in supporting the next frontier of technology, or a recruit eager to be part of a groundbreaking community—Hugging Face is the place for you.

Let’s build the future of AI together!

**Visit us at [Hugging Face](https://huggingface.co)**

--- 

**Follow us on:**  
GitHub | Twitter | LinkedIn | Discord

--- 

*Empowering the AI community, one model at a time.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation


In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
Hugging Face is an innovative platform dedicated to building the future of artificial intelligence (AI) through collaboration, community, and cutting-edge technology. With a mission to support the machine learning community, Hugging Face offers a dynamic environment where users can explore, create, and share models, datasets, and applications in AI.

---

## Our Offerings
- **Models**: Explore over 1 million machine learning models that cater to diverse applications, including text, image, audio, and even 3D challenges.
- **Datasets**: Access and share a vast repository of over 250,000 datasets, making data-driven AI development seamless.
- **Spaces**: Run applications built on Hugging Face's infrastructure, fostering collaboration among AI developers.
- **Enterprise Solutions**: Tailored for more than 50,000 organizations worldwide, providing advanced platforms with enterprise-grade security and dedicated support.

---

## A Community Focused on Collaboration
At Hugging Face, we believe in the power of collaboration. Our platform allows users to host and work together on public models and datasets, accelerating innovation in the AI space. We encourage everyone to create, discover, and contribute to machine learning projects, building a rich ecosystem for learners and experts alike.

---

## Our Customers
We are proud to serve a diverse clientele that includes some of the biggest names in technology:
- **Amazon Web Services**
- **Microsoft**
- **Google**
- **Meta**
- **Intel**

More than 50,000 enterprise organizations trust Hugging Face for their AI solutions.

---

## Company Culture
Hugging Face thrives on a culture of openness, inclusivity, and innovation. Our team is dedicated to maintaining a collaborative atmosphere that encourages creativity and knowledge sharing. We support our employees with a robust learning environment, empowering them to adopt the latest technologies and develop their skills further.

### Core Values:
- **Collaboration**: We believe community efforts lead to remarkable advancements.
- **Innovation**: Encouraging creative problem-solving and the exploration of new ideas.
- **Inclusivity**: Embracing diverse perspectives to enrich our platform and company.

---

## Careers at Hugging Face
Join our mission to build the future of AI! We are always on the lookout for passionate individuals who want to contribute to the field of machine learning. Whether you're interested in engineering, data science, marketing, or community engagement, we provide a supportive work environment and opportunities for growth.

**Explore our current openings and become part of a team that is shaping the future of AI**: [Careers at Hugging Face](https://huggingface.co/jobs)

---

## Connect With Us
Stay updated with our latest developments, community initiatives, and product offerings. Follow us on our social channels!
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

---

For more information about our services, products, or to get started, visit us at [Hugging Face](https://huggingface.co). Together, let's build the AI future!

In [28]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is a pioneering platform at the forefront of the AI community, dedicated to building the future through collaboration on models, datasets, and applications. We empower researchers, developers, and enterprises by providing a robust ecosystem where machine learning innovation thrives.

## Our Offerings
- **Models**: Access to over **1 million machine learning models** encompassing various applications and modalities, including text, image, video, audio, and 3D.
- **Datasets**: A comprehensive hub featuring more than **250,000 datasets** facilitating diverse research topics and projects.
- **Spaces**: Explore applications running on our platform to engage with advanced AI technology interactively.

### Feature Highlight: 
**Inference Providers on the Hub** - An exciting new addition facilitating easy integration of models from our Hub into various applications.

## Our Customers
Over **50,000 organizations** worldwide are leveraging the power of Hugging Face for their AI needs. Notable partners include:
- **Amazon Web Services (AWS)**
- **Google**
- **Microsoft**
- **Meta**

## Company Culture
At Hugging Face, we are committed to fostering a diverse, inclusive, and collaborative work environment where creativity and innovation are at the heart of what we do. Our community-oriented approach extends to both our employees and our users, encouraging knowledge sharing and mutual growth.

### Our Open Source Philosophy
We believe in building the **foundation of machine learning tools** through community collaboration. Our suite of open-source projects includes:
- **Transformers**: Industry-leading models for various ML tasks.
- **Diffusers**: Cutting-edge diffusion models for generating images and audio.
- **Tokenizers**: Fast and efficient solutions for managing text input.

## Careers at Hugging Face
We are continuously looking for talented individuals to join our team and contribute to meaningful projects in AI. We offer roles across various domains, including engineering, research, and product management, all in a dynamic and supportive environment.

### Why Join Us?
- Opportunity to work on groundbreaking technology and contribute to a vibrant community.
- Competitive compensation and benefits.
- A culture of learning and development with access to resources and support.

## Contact Us
**Join the AI revolution with Hugging Face**. Explore our [website](https://huggingface.co) to learn more about our solutions, community, and career opportunities, or connect with us on social media.

---
**Hugging Face – The AI community building the future.**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
</td>
</tr>

</table>


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>
